Base on: https://www.kdnuggets.com/2021/09/python-apis-data-science-project.html

In [1]:
#import libraries
import requests
import pandas as pd
import time

In [2]:
#keys
API_KEY = "AIXXXXXXXXXXXXXXXXXXXXXX"
CHANNEL_ID = "UCW8Ews7tdKKkBT6GdtQaXvQ"

## Response Analysis ##

In [5]:
pageToken = ''

url = "https://www.googleapis.com/youtube/v3/search?key=" + API_KEY + "&channelId=" + CHANNEL_ID + "&part=snippet,id&order=date&maxResults=10000" + pageToken

response = requests.get(url).json()

In [7]:
response

{'kind': 'youtube#searchListResponse',
 'etag': 'bMsdxaKceufsyv2XxxaH2I__z48',
 'nextPageToken': 'CDIQAA',
 'regionCode': 'RU',
 'pageInfo': {'totalResults': 80, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'BdQq8d0hljfXZeIGjJIISfivcNg',
   'id': {'kind': 'youtube#video', 'videoId': 'PlpUo6bHsBQ'},
   'snippet': {'publishedAt': '2021-08-09T02:18:01Z',
    'channelId': 'UCW8Ews7tdKKkBT6GdtQaXvQ',
    'title': 'Advanced Facebook Data Science SQL interview question [RANK()]',
    'description': 'This advanced SQL question is from the Facebook data science interview that tests your ability to write window functions and rank data and use some advanced ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/PlpUo6bHsBQ/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/PlpUo6bHsBQ/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/PlpUo6bHsBQ/hqd

#### Video data ####

In [12]:
video_id = response['items'][0]['id']['videoId']
video_id

'PlpUo6bHsBQ'

In [13]:
video_title = response['items'][0]['snippet']['title']
video_title = str(video_title).replace("&","")
video_title

'Advanced Facebook Data Science SQL interview question [RANK()]'

In [14]:
upload_date = response['items'][0]['snippet']['publishedAt']
upload_date

'2021-08-09T02:18:01Z'

In [15]:
upload_date = str(upload_date).split("T")[0]
upload_date

'2021-08-09'

#### Video stats ####

In [44]:
url = "https://www.googleapis.com/youtube/v3/videos?id=" + video_id + "&part=statistics&key=" + API_KEY
response = requests.get(url).json()

In [46]:
viewCount = response['items'][0]['statistics']['viewCount']
viewCount

'8270'

In [47]:
likeCount = response['items'][0]['statistics']['likeCount']
likeCount

'138'

In [48]:
dislikeCount = response['items'][0]['statistics']['dislikeCount']
dislikeCount

'4'

In [49]:
commentCount = response['items'][0]['statistics']['commentCount']
commentCount

'24'

## Make Functions to collect Data ##

In [27]:
def get_video_details(video_id):
    
    url = "https://www.googleapis.com/youtube/v3/videos?id=" + video_id + 9+"&part=statistics&key=" + API_KEY
    response = requests.get(url).json()
 
    return response['items'][0]['statistics']['viewCount'],\
            response['items'][0]['statistics']['likeCount'],\
            response['items'][0]['statistics']['dislikeCount'],\
            response['items'][0]['statistics']['commentCount']

In [39]:
def get_videos(df):
    
    pageToken = ""
    
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key=" + API_KEY + "&channelId=" + CHANNEL_ID +\
                "&part=snippet,id&order=date&maxResults=10000&" + pageToken
 
        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
       
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                #print(upload_date)
                view_count, like_count, dislike_count, comment_count = get_video_details(video_id)
                df = df.append({'Video_id': video_id, 'Video_title': video_title,
                                "Upload_date": upload_date, "View_count": view_count, 
                                "Like_count": like_count, "Dislike_count": dislike_count, 
                                "Comment_count":comment_count},
                                ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']
 
        except:
            break
            
    return df
    

In [41]:
#build our dataframe
df = pd.DataFrame(columns=["Video_id", "Video_title", "Upload_date",
                           "View_count", "Like_count", "Dislike_count",
                           "Comment_count"])

df = get_videos(df)

In [42]:
df.head()

,Video_id,Video_title,Upload_date,View_count,Like_count,Dislike_count,Comment_count
0,PlpUo6bHsBQ,Advanced Facebook Data Science SQL interview q...,2021-08-09,2826,106,0,27
1,uY2wfR8Dkqo,5 Tips for a Successful Data Science Interview,2021-07-26,3322,128,1,28
2,yY7yau9j3xk,Data Science SQL Scenario Based Interview Ques...,2021-07-20,2044,76,1,16
3,i-E4pdU2qXM,Data Science SQL Interview Question Walkthroug...,2021-07-13,1895,99,0,15
4,T1UhSuKqy3A,Most Common Data Science SQL Interview Questio...,2021-06-15,3499,141,0,30
